In [2]:
from datetime import datetime
from sqlalchemy import create_engine
import pymysql

In [7]:

import mysql.connector
import pandas as pd

def calcular_roi(simbolo, fecha_inicio, fecha_fin):
    # Conexión a la base de datos
    db_connection = mysql.connector.connect(
        host="127.0.0.1",
        user="root",  # Cambia por tu usuario de MySQL
        password="11jablum11",  # Cambia por tu contraseña de MySQL
        database="yfinance"
    )
    cursor = db_connection.cursor()



    # Consulta SQL para obtener el precio inicial y final en el periodo
    query = """
    SELECT fecha, precio_cierre
    FROM precios_historicos
    WHERE id_empresa = (SELECT id_empresa FROM empresas_sp500 WHERE simbolo = %s)
      AND fecha BETWEEN %s AND %s
    ORDER BY fecha ASC;
    """
    
    cursor.execute(query, (simbolo, fecha_inicio, fecha_fin))
    precios = cursor.fetchall()
    
    # Cerrar conexión
    cursor.close()
    db_connection.close()

    # Verifica si se obtuvieron precios en el rango de fechas
    if len(precios) < 2:
        st.warning("No se encontraron datos suficientes en el rango de fechas seleccionado.")
        return None

    # Precio inicial y final
    precio_inicial = precios[0][1]
    precio_final = precios[-1][1]
    
    # Calcular ROI
    roi = (precio_final - precio_inicial) / precio_inicial * 100

    return roi, pd.DataFrame(precios, columns=['Fecha', 'Precio Cierre'])




In [6]:
import pymysql

# Datos de conexión
host = 'pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com'
user = 'admin'
password = '11jablum11'
database = 'yfinance'
port = 3306

def calcular_roi(simbolo, fecha_compra, fecha_venta):
    try:
        # Conectar a la base de datos
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            port=port
        )
        cursor = connection.cursor()

        # Consulta para el precio de compra (precio de cierre en la fecha de compra)
        query_compra = """
        SELECT precio_cierre
        FROM precios_historicos
        WHERE id_empresa = %s AND fecha = %s
        """
        cursor.execute(query_compra, (simbolo, fecha_compra))
        resultado_compra = cursor.fetchone()

        # Consulta para el precio de venta (precio de cierre en la fecha de venta)
        query_venta = """
        SELECT precio_cierre
        FROM precios_historicos
        WHERE id_empresa = %s AND fecha = %s
        """
        cursor.execute(query_venta, (simbolo, fecha_venta))
        resultado_venta = cursor.fetchone()

        # Verificar que se encontraron precios en ambas fechas
        if resultado_compra and resultado_venta:
            precio_compra = resultado_compra[0]
            precio_venta = resultado_venta[0]

            # Calcular ROI
            ganancia = precio_venta - precio_compra
            roi = (ganancia / precio_compra) * 100

            print(f"Empresa: {simbolo}")
            print(f"Precio de compra ({fecha_compra}): {precio_compra}")
            print(f"Precio de venta ({fecha_venta}): {precio_venta}")
            print(f"ROI entre {fecha_compra} y {fecha_venta}: {round(roi, 2)}%")
        else:
            if not resultado_compra:
                print(f"No se encontró precio de cierre para {simbolo} en la fecha de compra {fecha_compra}.")
            if not resultado_venta:
                print(f"No se encontró precio de cierre para {simbolo} en la fecha de venta {fecha_venta}.")

    except pymysql.MySQLError as e:
        print("Error al conectar o ejecutar la consulta SQL:", e)

    finally:
        # Cerrar la conexión
        if connection:
            connection.close()

# Ejemplo de uso de la función
calcular_roi("MMM", "2020-01-02", "2020-12-31")  # Cambia "AAPL" y las fechas por los valores que necesitas verificar


No se encontró precio de cierre para MMM en la fecha de compra 2020-01-02.
No se encontró precio de cierre para MMM en la fecha de venta 2020-12-31.


In [9]:
calcular_roi("AAPL","2000-03-01","2001-04-06")

(-68.39896188489135,
           Fecha  Precio Cierre
 0    2000-03-01       0.982550
 1    2000-03-02       0.919875
 2    2000-03-03       0.965114
 3    2000-03-06       0.947679
 4    2000-03-07       0.926472
 ..          ...            ...
 274  2001-04-02       0.325576
 275  2001-04-03       0.305218
 276  2001-04-04       0.294058
 277  2001-04-05       0.314718
 278  2001-04-06       0.310496
 
 [279 rows x 2 columns])

In [10]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from funciones import obtener_cotizaciones  # Importar la función desde funciones.py

# Cargar datos de la base de datos
cotizaciones_df = obtener_cotizaciones()

# Crear la aplicación de JupyterDash
app = JupyterDash(__name__)

# Obtener la lista de empresas para el Dropdown
empresas = cotizaciones_df['Company'].unique()

# Layout del Dashboard
app.layout = html.Div([
    html.H1("Dashboard Financiero - Datos de Cotización"),
    html.Label("Seleccione la empresa:"),
    dcc.Dropdown(
        id='empresa-dropdown',
        options=[{'label': empresa, 'value': empresa} for empresa in empresas],
        value=empresas[0]  # Seleccionar la primera empresa como valor predeterminado
    ),
    dcc.Tabs([
        dcc.Tab(label='Precios Históricos', children=[
            dcc.Graph(id='precios-historicos')
        ]),
        dcc.Tab(label='Medias Móviles', children=[
            dcc.Graph(id='medias-moviles')
        ]),
        dcc.Tab(label='RSI', children=[
            dcc.Graph(id='rsi')
        ])
    ])
])

# Callback para actualizar Precios Históricos
@app.callback(
    Output('precios-historicos', 'figure'),
    Input('empresa-dropdown', 'value')
)
def actualizar_precios_historicos(empresa_seleccionada):
    # Filtramos los datos para la empresa seleccionada
    df = cotizaciones_df[cotizaciones_df['Company'] == empresa_seleccionada]
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Date'], y=df['Close'], mode='lines', name='Precio de Cierre'))
    fig.update_layout(title=f"Precios Históricos - {empresa_seleccionada}", xaxis_title="Fecha", yaxis_title="Precio de Cierre")
    return fig

# Callback para actualizar Medias Móviles
@app.callback(
    Output('medias-moviles', 'figure'),
    Input('empresa-dropdown', 'value')
)
def actualizar_medias_moviles(empresa_seleccionada):
    # Filtramos los datos para la empresa seleccionada y calculamos las medias móviles
    df = cotizaciones_df[cotizaciones_df['Company'] == empresa_seleccionada].copy()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['SMA_200'] = df['Close'].rolling(window=200).mean()
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Date'], y=df['Close'], mode='lines', name='Precio de Cierre'))
    fig.add_trace(go.Scatter(x=df['Date'], y=df['SMA_50'], mode='lines', name='SMA 50'))
    fig.add_trace(go.Scatter(x=df['Date'], y=df['SMA_200'], mode='lines', name='SMA 200'))
    fig.update_layout(title=f"Medias Móviles - {empresa_seleccionada}", xaxis_title="Fecha", yaxis_title="Precio")
    return fig

# Callback para actualizar RSI
@app.callback(
    Output('rsi', 'figure'),
    Input('empresa-dropdown', 'value')
)
def actualizar_rsi(empresa_seleccionada):
    # Filtramos los datos para la empresa seleccionada y calculamos el RSI
    df = cotizaciones_df[cotizaciones_df['Company'] == empresa_seleccionada].copy()
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Date'], y=df['RSI'], mode='lines', name='RSI'))
    fig.add_hline(y=70, line_dash="dash", line_color="red", annotation_text="Sobrecompra", annotation_position="top right")
    fig.add_hline(y=30, line_dash="dash", line_color="green", annotation_text="Sobreventa", annotation_position="bottom right")
    fig.update_layout(title=f"RSI (Relative Strength Index) - {empresa_seleccionada}", xaxis_title="Fecha", yaxis_title="RSI")
    return fig

# Ejecutar la aplicación en modo inline para Jupyter Notebook
app.run_server(mode="inline")


/var/folders/3l/2mfq68kd2m7_kpj_3qkpl1qw0000gn/T/ipykernel_1094/333576529.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/3l/2mfq68kd2m7_kpj_3qkpl1qw0000gn/T/ipykernel_1094/333576529.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/dash/dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [52]:
from funciones import obtener_cotizaciones

# Ejecuta la función y verifica si los datos se obtuvieron
cotizaciones_df = obtener_cotizaciones()

if cotizaciones_df.empty:
    print("No se obtuvieron datos de la base de datos. Verifica la conexión o consulta SQL.")
else:
    print("Datos obtenidos correctamente:")
    print(cotizaciones_df.head())



/Users/javier/Desktop/PFB/ProyectoPFB/Streamlit/funciones.py:114: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



No se obtuvieron datos de la base de datos. Verifica la conexión o consulta SQL.


In [53]:
import pymysql

try:
    # Conectar a la base de datos
    connection = pymysql.connect(
        host="pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com",
        user="admin",
        password="11jablum11",
        database="yfinance",
        port=3306
    )

    # Crear cursor
    cursor = connection.cursor()

    # Ejecutar una consulta simple
    cursor.execute("SELECT COUNT(*) FROM precios_historicos")
    count = cursor.fetchone()
    print(f"Total de registros en precios_historicos: {count[0]}")

    # Cerrar la conexión
    cursor.close()
    connection.close()

except pymysql.MySQLError as e:
    print(f"Error al conectar a la base de datos o ejecutar la consulta: {e}")


Total de registros en precios_historicos: 2756959


In [72]:
import pymysql
import pandas as pd

def obtener_cotizaciones():
    try:
        # Conectar a la base de datos
        connection = pymysql.connect(
            host="pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com",
            user="admin",
            password="11jablum11",
            database="yfinance",
            port=3306
        )

        # Crear un cursor
        cursor = connection.cursor()

        # Consulta SQL simplificada para verificar si obtenemos resultados de `precios_historicos` únicamente
        query = """
        SELECT fecha, precio_cierre
        FROM precios_historicos
        LIMIT 5
        """
        cursor.execute(query)
        rows = cursor.fetchall()

        # Cerrar la conexión
        cursor.close()
        connection.close()

        # Crear el DataFrame a partir de los datos obtenidos
        cotizaciones_df = pd.DataFrame(rows, columns=["Date", "Close"])
        
        return cotizaciones_df

    except pymysql.MySQLError as e:
        print(f"Error al conectar a la base de datos o ejecutar la consulta: {e}")
        return pd.DataFrame()


In [73]:
cotizaciones_df = obtener_cotizaciones()

if cotizaciones_df.empty:
    print("No se obtuvieron datos de la base de datos. Verifica la conexión o consulta SQL.")
else:
    print("Datos obtenidos correctamente:")
    print(cotizaciones_df.head())


Datos obtenidos correctamente:
         Date    Close
0  2000-01-03  19.8138
1  2000-01-04  19.0265
2  2000-01-05  19.5776
3  2000-01-06  21.1523
4  2000-01-07  21.5721


In [68]:
import pymysql
import pandas as pd

def obtener_cotizaciones():
    try:
        # Conectar a la base de datos
        connection = pymysql.connect(
            host="pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com",
            user="admin",
            password="11jablum11",
            database="yfinance",
            port=3306
        )

        # Crear un cursor
        cursor = connection.cursor()

        # Consulta SQL con `LEFT JOIN` para verificar la unión entre tablas
        query = """
        SELECT ph.fecha, ph.precio_cierre, es.nombre_empresa
        FROM precios_historicos ph
        LEFT JOIN empresas_sp500 es ON ph.id_empresa = es.id_empresa
        LIMIT 5
        """
        cursor.execute(query)
        rows = cursor.fetchall()

        # Cerrar la conexión
        cursor.close()
        connection.close()

        # Crear el DataFrame a partir de los datos obtenidos
        cotizaciones_df = pd.DataFrame(rows, columns=["Date", "Close", "Company"])
        
        return cotizaciones_df

    except pymysql.MySQLError as e:
        print(f"Error al conectar a la base de datos o ejecutar la consulta: {e}")
        return pd.DataFrame()


In [121]:
import pymysql
import pandas as pd

# Datos de conexión a la base de datos
host = 'pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com'
user = 'admin'
password = '11jablum11'
database = 'yfinance'
port = 3306

def obtener_cotizaciones_empresa(simbolo, fecha_inicio, fecha_fin):
    try:
        # Conectar a la base de datos
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            port=port
        )
        cursor = connection.cursor()

        # Consulta SQL para obtener las cotizaciones de la empresa en el rango de fechas
        query = """
        SELECT ph.fecha AS Date, ph.precio_cierre AS Close, es.nombre_empresa AS Company
        FROM precios_historicos ph
        INNER JOIN empresas_sp500 es ON ph.id_empresa = es.id_empresa
        WHERE es.simbolo = %s AND ph.fecha BETWEEN %s AND %s
        ORDER BY ph.fecha
        """
        
        # Ejecutar la consulta con los parámetros
        cursor.execute(query, (simbolo, fecha_inicio, fecha_fin))
        rows = cursor.fetchall()

        # Verificar si se obtuvieron resultados
        if rows:
            print(f"Primeros registros obtenidos: {rows[:5]}")  # Muestra los primeros 5 registros para depuración
        else:
            print("No se obtuvieron datos de la consulta SQL para el rango y símbolo especificado.")

        # Crear el DataFrame a partir de los datos obtenidos
        cotizaciones_df = pd.DataFrame(rows, columns=["Date", "Close", "Company"])

        # Cerrar la conexión
        cursor.close()
        connection.close()

        return cotizaciones_df

    except pymysql.MySQLError as e:
        print(f"Error al conectar a la base de datos o ejecutar la consulta: {e}")
        return pd.DataFrame()




In [122]:
# Ejemplo de uso de la función para una empresa y un rango de fechas
simbolo = "MMM"  # Cambia esto al símbolo de la empresa deseada
fecha_inicio = "2000-01-04"
fecha_fin = "2008-12-23"

cotizaciones_df = obtener_cotizaciones_empresa(simbolo, fecha_inicio, fecha_fin)

if cotizaciones_df.empty:
    print("No se obtuvieron datos de la base de datos para el símbolo y rango de fechas especificado.")
else:
    print("Datos obtenidos correctamente:")
    print(cotizaciones_df.head())



Primeros registros obtenidos: ((datetime.date(2000, 1, 4), 19.0265, '3M Company'), (datetime.date(2000, 1, 5), 19.5776, '3M Company'), (datetime.date(2000, 1, 6), 21.1523, '3M Company'), (datetime.date(2000, 1, 7), 21.5721, '3M Company'), (datetime.date(2000, 1, 10), 21.4672, '3M Company'))
Datos obtenidos correctamente:
         Date    Close     Company
0  2000-01-04  19.0265  3M Company
1  2000-01-05  19.5776  3M Company
2  2000-01-06  21.1523  3M Company
3  2000-01-07  21.5721  3M Company
4  2000-01-10  21.4672  3M Company


In [119]:
obtener_cotizaciones_empresa(simbolo, fecha_inicio, fecha_fin)

KeyboardInterrupt: 

In [118]:
import pymysql
import pandas as pd
from datetime import datetime, timedelta

# Datos de conexión a la base de datos
host = 'pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com'
user = 'admin'
password = '11jablum11'
database = 'yfinance'
port = 3306

def obtener_id_empresa(simbolo):
    """Obtener id_empresa basado en el símbolo de la empresa."""
    try:
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            port=port
        )
        cursor = connection.cursor()

        # Consulta para obtener el id_empresa basado en el símbolo
        query = "SELECT id_empresa FROM empresas_sp500 WHERE simbolo = %s"
        cursor.execute(query, (simbolo,))
        result = cursor.fetchone()

        cursor.close()
        connection.close()

        if result:
            return result[0]
        else:
            print(f"No se encontró id_empresa para el símbolo: {simbolo}")
            return None

    except pymysql.MySQLError as e:
        print(f"Error en la consulta para obtener id_empresa: {e}")
        return None

def obtener_cotizaciones_por_dia(id_empresa, fecha):
    """Obtener la cotización de cierre de una empresa en una fecha específica."""
    try:
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            port=port
        )
        cursor = connection.cursor()

        # Consulta SQL para obtener cotización de un solo día
        query = """
        SELECT fecha AS Date, precio_cierre AS Close
        FROM precios_historicos
        WHERE id_empresa = %s AND fecha = %s
        """
        cursor.execute(query, (id_empresa, fecha))
        result = cursor.fetchone()

        cursor.close()
        connection.close()

        # Si hay un resultado, devolverlo en formato de lista, si no, devolver None
        if result:
            return [result[0], result[1]]
        else:
            return None

    except pymysql.MySQLError as e:
        print(f"Error al conectar o ejecutar la consulta SQL: {e}")
        return None

def obtener_cotizaciones_empresa(simbolo, fecha_inicio, fecha_fin):
    """Obtener las cotizaciones de una empresa para un rango de fechas, día por día."""
    id_empresa = obtener_id_empresa(simbolo)
    if id_empresa is None:
        print("No se pudo obtener el id_empresa, verifique el símbolo de la empresa.")
        return pd.DataFrame()

    # Convertir fechas de inicio y fin en objetos datetime
    fecha_inicio_dt = datetime.strptime(fecha_inicio, "%Y-%m-%d")
    fecha_fin_dt = datetime.strptime(fecha_fin, "%Y-%m-%d")

    # Crear una lista para almacenar cada fila de cotización
    cotizaciones = []

    # Iterar sobre cada día en el rango de fechas
    fecha_actual = fecha_inicio_dt
    while fecha_actual <= fecha_fin_dt:
        # Convertir la fecha actual a string en formato YYYY-MM-DD
        fecha_str = fecha_actual.strftime("%Y-%m-%d")

        # Obtener la cotización para la fecha actual
        cotizacion = obtener_cotizaciones_por_dia(id_empresa, fecha_str)
        if cotizacion:
            cotizaciones.append(cotizacion)  # Agregar el resultado a la lista si existe

        # Incrementar la fecha en un día
        fecha_actual += timedelta(days=1)

    # Crear el DataFrame a partir de la lista de cotizaciones
    cotizaciones_df = pd.DataFrame(cotizaciones, columns=["Date", "Close"])

    return cotizaciones_df


In [117]:
import pymysql
import pandas as pd

# Datos de conexión a la base de datos
host = 'pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com'
user = 'admin'
password = '11jablum11'
database = 'yfinance'
port = 3306

def obtener_precios_empresa(simbolo):
    """Obtiene los precios diarios de apertura, cierre, máximo y mínimo de una única empresa, identificada por su símbolo."""
    try:
        # Conectar a la base de datos
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            port=port
        )
        cursor = connection.cursor()

        # Consulta SQL para unir `empresas_sp500` y `precios_historicos` para una empresa específica
        query = """
        SELECT es.simbolo AS Symbol, es.nombre_empresa AS Company, 
               ph.fecha AS Date, ph.precio_apertura AS Open, 
               ph.precio_cierre AS Close, ph.maximo AS High, 
               ph.minimo AS Low
        FROM precios_historicos ph
        INNER JOIN empresas_sp500 es ON ph.id_empresa = es.id_empresa
        WHERE es.simbolo = %s
        ORDER BY ph.fecha
        """
        
        # Ejecutar la consulta con el símbolo especificado
        cursor.execute(query, (simbolo,))
        rows = cursor.fetchall()

        # Crear el DataFrame directamente a partir de los datos obtenidos
        precios_df = pd.DataFrame(rows, columns=["Symbol", "Company", "Date", "Open", "Close", "High", "Low"])

        # Cerrar la conexión
        cursor.close()
        connection.close()

        # Verificar si se obtuvieron resultados
        if precios_df.empty:
            print(f"No se encontraron datos para la empresa con símbolo: {simbolo}")
        else:
            print(f"Datos obtenidos correctamente para la empresa con símbolo {simbolo}")

        return precios_df

    except pymysql.MySQLError as e:
        print(f"Error al conectar a la base de datos o ejecutar la consulta: {e}")
        return pd.DataFrame()



In [104]:
# Ejemplo de uso de la función para una empresa específica
simbolo = "AAPL"  # Cambia esto al símbolo que deseas probar
precios_empresa_df = obtener_precios_empresa(simbolo)

if precios_empresa_df.empty:
    print("No se obtuvieron datos para la empresa especificada.")
else:
    print("Datos obtenidos correctamente:")
    print(precios_empresa_df.head())



No se encontraron datos para la empresa con símbolo: AAPL
No se obtuvieron datos para la empresa especificada.


In [105]:
import pymysql

def obtener_id_empresa(simbolo):
    """Obtiene el id_empresa para un símbolo específico de empresas_sp500."""
    try:
        connection = pymysql.connect(
            host='pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com',
            user='admin',
            password='11jablum11',
            database='yfinance',
            port=3306
        )
        cursor = connection.cursor()

        # Consulta para obtener el id_empresa para el símbolo
        query = "SELECT id_empresa FROM empresas_sp500 WHERE simbolo = %s"
        cursor.execute(query, (simbolo,))
        result = cursor.fetchone()

        cursor.close()
        connection.close()

        # Verificar si se obtuvo un resultado
        if result:
            id_empresa = result[0]
            print(f"id_empresa para el símbolo '{simbolo}': {id_empresa}")
            return id_empresa
        else:
            print(f"No se encontró id_empresa para el símbolo: {simbolo}")
            return None

    except pymysql.MySQLError as e:
        print("Error al conectar o ejecutar la consulta SQL:", e)
        return None

# Prueba para ver si podemos obtener el id_empresa para el símbolo
simbolo = "AAPL"  # Cambia al símbolo que quieras probar
id_empresa = obtener_id_empresa(simbolo)


id_empresa para el símbolo 'AAPL': 100


In [111]:
def consultar_precio_cierre_simple(id_empresa, fecha):
    """Consulta el precio de cierre en una fecha específica para un id_empresa dado."""
    try:
        # Conectar a la base de datos
        connection = pymysql.connect(
            host='pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com',
            user='admin',
            password='11jablum11',
            database='yfinance',
            port=3306
        )
        cursor = connection.cursor()

        # Consulta simple para verificar datos en precios_historicos usando id_empresa
        query = """
        SELECT precio_cierre
        FROM precios_historicos
        WHERE id_empresa = %s AND fecha = %s
        """
        cursor.execute(query, (id_empresa, fecha))
        resultado = cursor.fetchone()

        # Verificar si se obtuvo un precio de cierre
        if resultado:
            precio_cierre = resultado[0]
            print(f"Precio de cierre para el id_empresa {id_empresa} en la fecha {fecha}: {precio_cierre}")
        else:
            print(f"No se encontró precio de cierre para el id_empresa {id_empresa} en la fecha {fecha}.")

    except pymysql.MySQLError as e:
        print("Error al conectar o ejecutar la consulta SQL:", e)

    finally:
        # Cerrar la conexión
        if connection:
            connection.close()

# Ejemplo de uso de la función
fecha = "2000-06-05"
if id_empresa:
    consultar_precio_cierre_simple(id_empresa, fecha)


No se encontró precio de cierre para el id_empresa 100 en la fecha 2000-06-05.


In [112]:
def listar_fechas_disponibles(id_empresa):
    """Lista las fechas disponibles en precios_historicos para un id_empresa específico."""
    try:
        connection = pymysql.connect(
            host='pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com',
            user='admin',
            password='11jablum11',
            database='yfinance',
            port=3306
        )
        cursor = connection.cursor()

        # Consulta para listar las fechas disponibles para el id_empresa
        query = """
        SELECT fecha
        FROM precios_historicos
        WHERE id_empresa = %s
        ORDER BY fecha
        """
        cursor.execute(query, (id_empresa,))
        fechas = cursor.fetchall()

        cursor.close()
        connection.close()

        # Mostrar las fechas obtenidas
        if fechas:
            print(f"Fechas disponibles para id_empresa {id_empresa}:")
            for fecha in fechas[:10]:  # Muestra las primeras 10 fechas como ejemplo
                print(fecha[0])
        else:
            print(f"No se encontraron fechas para el id_empresa {id_empresa}.")

    except pymysql.MySQLError as e:
        print("Error al conectar o ejecutar la consulta SQL:", e)

# Ejemplo de uso de la función
if id_empresa:
    listar_fechas_disponibles(id_empresa)


No se encontraron fechas para el id_empresa 100.


In [116]:
def listar_id_empresa_precios_historicos():
    """Lista todos los id_empresa disponibles en precios_historicos para verificar consistencia."""
    try:
        connection = pymysql.connect(
            host='pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com',
            user='admin',
            password='11jablum11',
            database='yfinance',
            port=3306
        )
        cursor = connection.cursor()

        # Consulta para obtener todos los id_empresa en precios_historicos
        query = "SELECT DISTINCT id_empresa FROM precios_historicos"
        cursor.execute(query)
        ids = cursor.fetchall()

        cursor.close()
        connection.close()

        # Mostrar los id_empresa obtenidos
        print("id_empresa disponibles en precios_historicos:")
        for id_empresa in ids:
            print(id_empresa[0])

    except pymysql.MySQLError as e:
        print("Error al conectar o ejecutar la consulta SQL:", e)

# Llamada a la función para listar los id_empresa en precios_historicos
listar_id_empresa_precios_historicos()


id_empresa disponibles en precios_historicos:
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
122
123
124
125
126
127
128
129
130
131
132
133
134
135
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
3

In [115]:
def comparar_id_empresa():
    """Compara los id_empresa entre empresas_sp500 y precios_historicos."""
    try:
        connection = pymysql.connect(
            host='pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com',
            user='admin',
            password='11jablum11',
            database='yfinance',
            port=3306
        )
        cursor = connection.cursor()

        # Obtener todos los id_empresa de empresas_sp500
        cursor.execute("SELECT DISTINCT id_empresa FROM empresas_sp500")
        ids_empresas = set([row[0] for row in cursor.fetchall()])

        # Obtener todos los id_empresa de precios_historicos
        cursor.execute("SELECT DISTINCT id_empresa FROM precios_historicos")
        ids_precios = set([row[0] for row in cursor.fetchall()])

        cursor.close()
        connection.close()

        # Comparar los id_empresa en ambas tablas
        print("id_empresa en empresas_sp500 que no están en precios_historicos:", ids_empresas - ids_precios)
        print("id_empresa en precios_historicos que no están en empresas_sp500:", ids_precios - ids_empresas)

    except pymysql.MySQLError as e:
        print("Error al conectar o ejecutar la consulta SQL:", e)

# Llamada a la función para comparar id_empresa entre tablas
comparar_id_empresa()


id_empresa en empresas_sp500 que no están en precios_historicos: {136, 121}
id_empresa en precios_historicos que no están en empresas_sp500: set()


In [1]:
from funciones import obtener_cotizaciones

In [2]:
df=obtener_cotizaciones()

In [3]:
df.tail()

,Date,Close,Company
2756954,2024-10-24,181.50,Zoetis Inc.
2756955,2024-10-25,180.01,Zoetis Inc.
2756956,2024-10-28,182.76,Zoetis Inc.
2756957,2024-10-29,181.27,Zoetis Inc.
2756958,2024-10-30,182.74,Zoetis Inc.
